
## Capstone Project "What's Happening in my Neighborhood"
by Lori Butler  

**Data Questions**
1. Which neighborhoods have had the most, and the least, growth during the past three years based on the count of building permits by type, and by cost of construction.
2. Do any neighborhoods have recent increases in building permit applications which may signal growth in the near future?
3. In instances where zoning changes are sought, might those be a leading indicator of a subsequent increase in building permit applications? If so, what is the average time lag?


In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd   # Prerequisite: Activate geospatial environment via Conda Prompt
import matplotlib.pyplot as plt  
import folium                   
from folium.plugins import MarkerCluster
from shapely.geometry import Point  

# To create rolling dates to view past week, 30 days, etc.  (Not sure if I need this yet)
from datetime import timedelta  

In [2]:
# Building Permit Applications
# Expecting: 3106 rows, 21 column

df_bldg_apps_clean = pd.read_csv('../data/cleaned/bldg_permit_applications_clean.csv')
print(df_bldg_apps_clean.shape)
df_bldg_apps_clean.head(1)

(3106, 21)


,permit_number,permit_type_descr,permit_subtype_descr,parcel,date_entered,construction_cost,address,city,state,zip,...,contact,permit_type,permit_subtype,ivr_tracking_number,purpose,council_district,mapped_location,lat,lon,category
0,T2020034761,Building Residential - New,Single Family Residence,08211018000,2020-06-04,NaN,518 N 2ND ST,NASHVILLE,TN,37207,...,EASTEND CONTRACTING LLC,CARN,CAA01R301,3830078,to construct 2034SF single family residence. 5...,5.0,"518 N 2ND ST\nNASHVILLE, TN 37207\n(36.18049, ...",36.18049,-86.771778,Residential


In [3]:
# Building Dept. Permits Issued
# Expecting: 33909 rows, 23 columns

# low_memory = False was added to remove a low-memory warning. Doing this prevents the
# system from trying to assign dtypes until after the full file has been read
# Resource: https://tinyurl.com/stackoverflow-low-memory

df_bldg_issued_clean = pd.read_csv('../data/cleaned/bldg_permits_issued_clean.csv'
                                   , low_memory = False)
print(df_bldg_issued_clean.shape)
df_bldg_issued_clean.head(1)

(33909, 23)


,permit_number,permit_type_descr,permit_subtype_descr,parcel,date_entered,date_issued,construction_cost,address,city,state,...,permit_type,permit_subtype,ivr_tracking_number,purpose,council_district,census_tract,mapped_location,lat,lon,category
0,2020032857,Building Commercial Rehab Storm Damage,"Multifamily, Condominium 3&4 Unit Bldg",08214005500,2020-05-27,2020-06-04,800000.0,186 N 1ST ST,NASHVILLE,TN,...,CACL,CAA03R298,3824603,",there is some remodeling to be performed in t...",5.0,37019300.0,"186 N 1ST ST\nNASHVILLE, TN 37213\n(36.173878,...",36.173878,-86.774064,Commercial


In [4]:
# Planning Dept. Applications and Issued Permits
# Expecting: 520 rows, 25 columns

df_planning_clean = pd.read_csv('../data/cleaned/planning_dept_clean.csv')
print(df_planning_clean.shape)
df_planning_clean.head(1)

(520, 25)


,date_submitted,application_type_descr,mpc_case_number,ordinance_number,status,mpc_meeting_date,mpc_action,project_name,location,reviewer,...,applicant_phone,applicant_address_1,applicant_address_2,applicant_city,applicant_state,applicant_zip,council_district,latitude,longitude,mapped_location
0,2018-01-17,Specific Plan (New),2018SP-027-001,BL2020-221,NEW,2020-07-23,NaN,12558 OLD HICKORY BLVD,12558 OLD HICKORY BLVD 37013,Latisha Birkeland,...,615-297-5166,516 HEATHER PLACE,NaN,NASHVILLE,TN,37204,33 (Antoinette Lee),36.03535,-86.604835,"(36.035350452000046, -86.60483534099995)"


In [5]:
# Geopandas, read shapefile:  2
# Expecting: 288 rows, 2 columns

df_na_bound_clean = gpd.read_file('../data/cleaned/neighborhood_association_boundaries_clean.shp')
print(df_na_bound_clean.shape)
df_na_bound_clean.head(1)

(288, 2)


,name,geometry
0,Historic Buena Vista,"POLYGON ((-86.79511 36.17576, -86.79403 36.176..."
